In [ ]:
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 237 kB 5.2 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import csv
from tensorflow.keras.layers import TextVectorization
import tensorflow_model_optimization as tfmot

In [ ]:
with open("eng-ind.csv") as f:
    csvreader = csv.reader(f, delimiter=",")
    next(csvreader)
    eng_ind, label = list(),list()
    for i in csvreader:
        eng_ind.append(i[0])
        label.append(0)
        eng_ind.append(i[1])
        label.append(1)
        
vectorize = TextVectorization(output_sequence_length=32)
vectorize.adapt(tf.data.Dataset.from_tensor_slices(eng_ind))


In [ ]:
datas = list()
for i in eng_ind:
  datas.append(vectorize(i))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((datas,label)).batch(8)

In [ ]:
train = dataset.take(int(0.9*len(dataset)))
val = dataset.skip(int(0.9*len(dataset)))

In [ ]:
len(vectorize.get_vocabulary())

8838

In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer

In [ ]:
model = tf.keras.Sequential()
model.add(quantize_annotate_layer(layers.Embedding(input_dim=1000, output_dim=64, input_shape=(32,))))
model.add(layers.Flatten())

model.add(layers.Dense(64, activation="tanh"))
model.add(layers.Dense(64,activation="tanh"))
model.add(layers.Dense(32,activation="tanh"))

model.add(layers.Dense(1, activation="sigmoid"))

model.compile(
    loss = "binary_crossentropy",
    optimizer = "RMSprop",
    metrics = ["accuracy"]
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_annotate_4 (Quanti  (None, 32, 64)           64000     
 zeAnnotate)                                                     
                                                                 
 flatten_4 (Flatten)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 64)                131136    
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 1)                 33        
                                                      

In [ ]:
model.fit(train,
          validation_data=val,
          epochs=5)

Epoch 1/5
1984/1984 [==============================] - 9s 4ms/step - loss: 0.0487 - accuracy: 0.9779 - val_loss: 0.0139 - val_accuracy: 0.9977
Epoch 2/5
1984/1984 [==============================] - 8s 4ms/step - loss: 0.0183 - accuracy: 0.9937 - val_loss: 0.0116 - val_accuracy: 0.9972
Epoch 3/5
1984/1984 [==============================] - 9s 4ms/step - loss: 0.0186 - accuracy: 0.9941 - val_loss: 0.0203 - val_accuracy: 0.9955
Epoch 4/5
1984/1984 [==============================] - 9s 4ms/step - loss: 0.0198 - accuracy: 0.9950 - val_loss: 0.0171 - val_accuracy: 0.9972
Epoch 5/5
1984/1984 [==============================] - 9s 4ms/step - loss: 0.0175 - accuracy: 0.9945 - val_loss: 0.0070 - val_accuracy: 0.9977


## Predict

## Greater than 0.5 mean Indonesian language, less than 0.5 mean English language.

In [ ]:
teks = tf.constant(["i love you",
               "When using this layer as the first layer in a model, provide an input_shape argument (tuple of integers or None, e.g. (10, 128) for sequences of 10 vectors of 128-dimensional vectors, or (None, 128) for variable-length sequences of 128-dimensional vectors.",
               "Indonesia, dengan nama resmi Republik Indonesia atau lengkapnya Negara Kesatuan Republik Indonesia, adalah sebuah negara kepulauan di Asia Tenggara yang dilintasi garis khatulistiwa dan berada di antara daratan benua Asia dan Oseania, serta antara Samudra Pasifik dan Samudra Hindia"])


model.predict(vectorize(teks))

array([[4.8646567e-09],
       [5.0633657e-09],
       [1.0000000e+00]], dtype=float32)

## Save model into tensorflow format

In [ ]:
model.save("model")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


## Convert tensorflow model to tflite

In [7]:
converter = tf.lite.TFLiteConverter.from_saved_model("model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("model/model.tflite","wb") as f:
    f.write(tflite_model)

##Parse tokenizer into json

In [ ]:
import json

word_index = {}
# index_word = {}

for i,j in enumerate(vectorize.get_vocabulary()):
  word_index[j] = i
  # index_word[i] = j

with open("word_index.json","w") as f:
  json.dump(word_index,f,indent=4)

# with open("index_word.json","w") as f:
#   json.dump(word_index,f,indent=4)

##Predict Using tflite and keras tokenizer

In [111]:
teks = tf.constant(["i love you",
               "When using this layer as the first layer in a model, provide an input_shape argument (tuple of integers or None, e.g. (10, 128) for sequences of 10 vectors of 128-dimensional vectors, or (None, 128) for variable-length sequences of 128-dimensional vectors.",
               "Indonesia, dengan nama resmi Republik Indonesia atau lengkapnya Negara Kesatuan Republik Indonesia, adalah sebuah negara kepulauan di Asia Tenggara yang dilintasi garis khatulistiwa dan berada di antara daratan benua Asia dan Oseania, serta antara Samudra Pasifik dan Samudra Hindia",
               "hi kami dari red team",
               "hi we're from red team",
               "mantab pak eko",
               "lu cakep juga yak"])


In [103]:
interpreter = tf.lite.Interpreter(model_path="model/model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# must have 2 dimension and float32 dtype
for i in range(len(teks)):
  input_data = np.array([vectorize(teks[i])], dtype=np.float32)

  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()

  predictions = interpreter.get_tensor(output_details[0]['index'])

  print(teks[i])
  print(predictions)

tf.Tensor(b'i love you', shape=(), dtype=string)
[[4.86385e-09]]
tf.Tensor(b'When using this layer as the first layer in a model, provide an input_shape argument (tuple of integers or None, e.g. (10, 128) for sequences of 10 vectors of 128-dimensional vectors, or (None, 128) for variable-length sequences of 128-dimensional vectors.', shape=(), dtype=string)
[[5.0901594e-09]]
tf.Tensor(b'Indonesia, dengan nama resmi Republik Indonesia atau lengkapnya Negara Kesatuan Republik Indonesia, adalah sebuah negara kepulauan di Asia Tenggara yang dilintasi garis khatulistiwa dan berada di antara daratan benua Asia dan Oseania, serta antara Samudra Pasifik dan Samudra Hindia', shape=(), dtype=string)
[[1.]]
tf.Tensor(b'hi kami dari red team', shape=(), dtype=string)
[[1.]]
tf.Tensor(b"hi we're from red team", shape=(), dtype=string)
[[4.921822e-09]]
tf.Tensor(b'mantab pak eko', shape=(), dtype=string)
[[1.10458146e-07]]
tf.Tensor(b'lu cakep juga yak', shape=(), dtype=string)
[[0.99999994]]


##Predict with tflite and json instead of tokenizer

In [39]:
import json

with open("model/word_index.json") as f:
  kamus = json.load(f)

In [123]:
def tf_lower_and_split_punct(text):
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, "[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", '')

  text = tf.strings.strip(text)
  text = tf.strings.split(text)
  return text

def sequenize(teks):
  hasil = list()
  counter = 32
  for i in teks:
    if i.numpy().decode() in kamus.keys():
      hasil.append(kamus[i.numpy().decode()])
      counter -= 1
      if counter == 0:
        return np.array([hasil], dtype=np.float32)
    else:
      hasil.append(1)
      counter -= 1
      if counter == 0:
        return np.array([hasil], dtype=np.float32)

  for i in range(counter):
    hasil.append(0)

  return tf.constant([hasil])

In [82]:
test_uwu = "saya su'ka kamu, uwu!!!!"
tf_lower_and_split_punct(test_uwu).numpy(), len(tf_lower_and_split_punct(test_uwu))

(array([b'saya', b'suka', b'kamu', b'uwu'], dtype=object), 4)

In [118]:
uwu = sequenize(tf_lower_and_split_punct(test_uwu))
uwu

<tf.Tensor: shape=(1, 32), dtype=int32, numpy=
array([[16, 84, 12,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int32)>

In [119]:
np.array([vectorize(teks[0])], dtype=np.float32)

array([[  4., 326.,   3.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]],
      dtype=float32)

In [122]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# must have 2 dimension and float32 dtype
for i in teks:
  input_data = np.array(sequenize(tf_lower_and_split_punct(i)), dtype=np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()

  predictions = interpreter.get_tensor(output_details[0]['index'])

  print(i.numpy().decode())
  print(predictions)

i love you
[[4.86385e-09]]
When using this layer as the first layer in a model, provide an input_shape argument (tuple of integers or None, e.g. (10, 128) for sequences of 10 vectors of 128-dimensional vectors, or (None, 128) for variable-length sequences of 128-dimensional vectors.
[[5.0743685e-09]]
Indonesia, dengan nama resmi Republik Indonesia atau lengkapnya Negara Kesatuan Republik Indonesia, adalah sebuah negara kepulauan di Asia Tenggara yang dilintasi garis khatulistiwa dan berada di antara daratan benua Asia dan Oseania, serta antara Samudra Pasifik dan Samudra Hindia
[[1.]]
hi kami dari red team
[[1.]]
hi we're from red team
[[4.9175903e-09]]
mantab pak eko
[[0.10529777]]
lu cakep juga yak
[[0.99999994]]


##Download All file needed to do inference in kotlin

In [125]:
!zip -r /content/file.zip model

updating: model/ (stored 0%)
updating: model/saved_model.pb (deflated 89%)
updating: model/assets/ (stored 0%)
updating: model/variables/ (stored 0%)
updating: model/variables/variables.data-00000-of-00001 (deflated 33%)
updating: model/variables/variables.index (deflated 64%)
updating: model/keras_metadata.pb (deflated 90%)
  adding: model/word_index.json (deflated 69%)
  adding: model/model.tflite (deflated 21%)


In [126]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Vanny Ezhaan Nur Sandika